In [1]:
%matplotlib inline
import torch
from torch import nn
from IPython import display
import torchvision
from torch.utils import data
from torchvision import transforms
import matplotlib.pyplot as plt

In [2]:
#读取数据并且返回训练集和测试集
def load_data_fashion_mnist(batch_size, resize=None):  #@save
    """下载Fashion-MNIST数据集，然后将其加载到内存中"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="D:/书籍资料整理/torch_data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="D:/书籍资料整理/torch_data", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=2),
            data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=2))

def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater):  #@save
    """训练模型（定义见第3章）"""
    for epoch in range(num_epochs):
        #每迭代周期
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
        #每迭代周期的损失函数
        test_acc = evaluate_accuracy(net, test_iter)
        print('迭代次数:',epoch+1,',训练集损失:',train_metrics[0],',训练集精度:',train_metrics[1],',测试集准确率:',test_acc)
    train_loss, train_acc = train_metrics
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc

def train_epoch_ch3(net, train_iter, loss, updater):  #@save
    """训练模型一个迭代周期（定义见第3章）"""
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用PyTorch内置的优化器和损失函数
            updater.zero_grad()
            l.sum().backward()
            updater.step()
        else:
            # 使用定制的优化器和损失函数
            #反向传播
            l.sum().backward()
            #梯度下降并且更新参数
            updater(X.shape[0])
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    # 返回训练损失和训练精度
    return metric[0] / metric[2], metric[1] / metric[2]

class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    
def accuracy(y_hat, y):  #@save
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

def evaluate_accuracy(net, data_iter):  #@save
    """计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [3]:
net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

In [4]:
batch_size, lr, num_epochs = 256, 0.1, 10
loss = nn.CrossEntropyLoss()
trainer = torch.optim.SGD(net.parameters(), lr=lr)

train_iter, test_iter = load_data_fashion_mnist(batch_size)
train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

迭代次数: 1 ,训练集损失: 0.0041033316413561505 ,训练集精度: 0.6379833333333333 ,测试集准确率: 0.7056
迭代次数: 2 ,训练集损失: 0.0023610575978954633 ,训练集精度: 0.7867833333333333 ,测试集准确率: 0.8026
迭代次数: 3 ,训练集损失: 0.002030052250623703 ,训练集精度: 0.81855 ,测试集准确率: 0.7902
迭代次数: 4 ,训练集损失: 0.0018820811132589975 ,训练集精度: 0.8318166666666666 ,测试集准确率: 0.8216
迭代次数: 5 ,训练集损失: 0.0017794992471734682 ,训练集精度: 0.8401166666666666 ,测试集准确率: 0.8188
迭代次数: 6 ,训练集损失: 0.0017006190260251363 ,训练集精度: 0.8473 ,测试集准确率: 0.8391
迭代次数: 7 ,训练集损失: 0.0016291755591829617 ,训练集精度: 0.8532 ,测试集准确率: 0.8394
迭代次数: 8 ,训练集损失: 0.0015757244234283765 ,训练集精度: 0.85745 ,测试集准确率: 0.8433
迭代次数: 9 ,训练集损失: 0.001537490685780843 ,训练集精度: 0.8603333333333333 ,测试集准确率: 0.8477
迭代次数: 10 ,训练集损失: 0.0014895920264224212 ,训练集精度: 0.8656833333333334 ,测试集准确率: 0.7962
